## Śledzenie twarzy

### Ręczne śledzenie twarzy, wybranie kilku punktów

In [1]:
from moviepy.editor import *
from moviepy.video.tools.tracking import manual_tracking, Trajectory
import numpy as np
import cv2

In [17]:
clip = VideoFileClip("clips/chaplin_cut.mp4")
clip = clip.subclip(3,10)

In [20]:
# manualne "trackowanie/śledzenie" obiektu i zapis współrzendych do pliku
# liczba fps klipu (np.24) powinna być podzielna przez liczbę fps trackowania (np.6). 
# Im więcej fps trackowania, tym częstszy zapis współrzędnych

# manual_tracking(clip,fps=6,nobjects=1, savefile="trackings/track_chaplin2.txt")

In [22]:
def object_tracked(get_frame, t):
    global circle_radius
    global iterator, traj1, clip_fps
    global skipped_frame, num_of_skipped_frames
    frame = get_frame(t)
    new_image = frame.copy()
    
    x = None
    y = None
    tt = None
    center_coordinates = None
    
    if(iterator >= len(traj1.tt)):
        x = traj1.xx[iterator-1]
        y = traj1.yy[iterator-1]
        center_coordinates = (int(x), int(y))
    else:
        x = traj1.xx[iterator]
        y = traj1.yy[iterator]
        tt = traj1.tt[iterator]

    
    if center_coordinates == None:
        t_calc = round(abs(t-tt),2)
        if (t_calc < 0.02):
            center_coordinates = (int(x), int(y))
            skipped_frame = None
            num_of_skipped_frames = None
            iterator+=1
        else:
            prev_t = traj1.tt[iterator-1]
            prev_x = traj1.xx[iterator-1]
            prev_y = traj1.yy[iterator-1]

            if skipped_frame == None:
                num_of_skipped_frames = round((((tt-prev_t)*clip_fps)), 2)
                skipped_frame = 1

            shift_x = (x - prev_x)/num_of_skipped_frames * skipped_frame
            shift_y = (y - prev_y)/num_of_skipped_frames * skipped_frame
            current_x = prev_x + shift_x
            current_y = prev_y + shift_y

            skipped_frame += 1
            center_coordinates = (int(current_x), int(current_y))
        
    circle_radius = 100
    color = (255, 0, 0)
    thickness = 10
    new_image = cv2.circle(new_image, center_coordinates, circle_radius, color, thickness)
    print(t,center_coordinates)
    return new_image

clip_fps = clip.fps
skipped_frame = None
num_of_skipped_frames = None
iterator = 0
traj1, = Trajectory.load_list('trackings/track_chaplin2.txt')

clip_blurred = clip.fl(object_tracked)

#clip_blurred.ipython_display(loop=True, autoplay=True, rate=5000, width = 400)

In [23]:
#clip_blurred.write_videofile("output/tracking_object.mp4")

t:   2%|█▏                                                                   | 3/168 [00:00<00:06, 26.10it/s, now=None]

Moviepy - Building video output/tracking_object.mp4.
Moviepy - Writing video output/tracking_object.mp4



Moviepy - Done !
Moviepy - video ready output/tracking_object.mp4


In [21]:
clip.close()
clip_blurred.close()

### Zamazywanie wykrytej twarzy

In [4]:
clip = VideoFileClip("clips/chaplin_cut.mp4")
clip = clip.subclip(3,10)

In [6]:
# manualne "trackowanie/śledzenie" obiektu i zapis współrzendych do pliku
# liczba fps klipu (np.24) powinna być podzielna przez liczbę fps trackowania (np.6). 
# Im więcej fps trackowania, tym częstszy zapis współrzędnych
#manual_tracking(clip,fps=6,nobjects=1, savefile="trackings/track_chaplin2.txt")

In [ ]:
def blur_tracked(get_frame, t):
    global circle_radius
    global iterator, traj1, clip_fps
    global skipped_frame, num_of_skipped_frames
    frame = get_frame(t)
    new_image = frame.copy()
    
    x = None
    y = None
    tt = None
    center_coordinates = None
    
    if(iterator >= len(traj1.tt)):
        x = traj1.xx[iterator-1]
        y = traj1.yy[iterator-1]
        center_coordinates = (int(x), int(y))
    else:
        x = traj1.xx[iterator]
        y = traj1.yy[iterator]
        tt = traj1.tt[iterator]

    
    if center_coordinates == None:
        t_calc = round(abs(t-tt),2)
        if (t_calc < 0.02):
            center_coordinates = (int(x), int(y))
            skipped_frame = None
            num_of_skipped_frames = None
            iterator+=1
        else:
            prev_t = traj1.tt[iterator-1]
            prev_x = traj1.xx[iterator-1]
            prev_y = traj1.yy[iterator-1]

            if skipped_frame == None:
                num_of_skipped_frames = round((((tt-prev_t)*clip_fps)), 2)
                skipped_frame = 1

            shift_x = (x - prev_x)/num_of_skipped_frames * skipped_frame
            shift_y = (y - prev_y)/num_of_skipped_frames * skipped_frame
            current_x = prev_x + shift_x
            current_y = prev_y + shift_y

            skipped_frame += 1
            center_coordinates = (int(current_x), int(current_y))
        
    circle_radius = 100
    color = (255, 255, 255)
    thickness = -1
    height,width,depth = new_image.shape
    mask = np.zeros((height,width,depth), np.uint8)
    mask = cv2.circle(mask, center_coordinates, circle_radius,(255,255,255), -1)
    blur_img = cv2.blur(new_image,(25,25))
    final_img = np.where(mask > 0, blur_img, new_image)
    return final_img

# Utworzenie zmiennych, które będą używane w metodzie

clip_fps = clip.fps
skipped_frame = None
num_of_skipped_frames = None
iterator = 0
traj1, = Trajectory.load_list('trackings/track_chaplin2.txt')

clip_blurred = clip.fl(blur_tracked)

clip_blurred.ipython_display(loop=True, autoplay=True, rate=5000, width = 400)

In [27]:
#clip_blurred.write_videofile("output/tracking_blurred_object.mp4")

chunk:   1%|▊                                                                | 2/155 [00:00<00:09, 16.66it/s, now=None]

Moviepy - Building video output/tracking_blurred_object.mp4.
MoviePy - Writing audio in tracking_blurred_objectTEMP_MPY_wvf_snd.mp3


t:   2%|█▋                                                                   | 4/168 [00:00<00:04, 34.96it/s, now=None]

MoviePy - Done.
Moviepy - Writing video output/tracking_blurred_object.mp4



Moviepy - Done !
Moviepy - video ready output/tracking_blurred_object.mp4


In [29]:
clip.close()
clip_blurred.close()

### Test dla trackingu dla fps = 1

In [2]:
clip = VideoFileClip("clips/chaplin_cut.mp4")
clip = clip.subclip(3,10)

In [32]:
# manualne "trackowanie/śledzenie" obiektu i zapis współrzendych do pliku
# liczba fps klipu (np.24) powinna być podzielna przez liczbę fps trackowania (np.6). 
# Im więcej fps trackowania, tym częstszy zapis współrzędnych

#manual_tracking(clip,fps=1,nobjects=1, savefile="trackings/track_chaplin3.txt")

In [ ]:
def object_tracked(get_frame, t):
    global circle_radius
    global iterator, traj1, clip_fps
    global skipped_frame, num_of_skipped_frames
    frame = get_frame(t)
    new_image = frame.copy()
    
    x = None
    y = None
    tt = None
    center_coordinates = None
    
    if(iterator >= len(traj1.tt)):
        x = traj1.xx[iterator-1]
        y = traj1.yy[iterator-1]
        center_coordinates = (int(x), int(y))
    else:
        x = traj1.xx[iterator]
        y = traj1.yy[iterator]
        tt = traj1.tt[iterator]

    
    if center_coordinates == None:
        t_calc = round(abs(t-tt),2)
        if (t_calc < 0.02):
            center_coordinates = (int(x), int(y))
            skipped_frame = None
            num_of_skipped_frames = None
            iterator+=1
        else:
            prev_t = traj1.tt[iterator-1]
            prev_x = traj1.xx[iterator-1]
            prev_y = traj1.yy[iterator-1]

            if skipped_frame == None:
                num_of_skipped_frames = round((((tt-prev_t)*clip_fps)), 2)
                skipped_frame = 1

            shift_x = (x - prev_x)/num_of_skipped_frames * skipped_frame
            shift_y = (y - prev_y)/num_of_skipped_frames * skipped_frame
            current_x = prev_x + shift_x
            current_y = prev_y + shift_y

            skipped_frame += 1
            center_coordinates = (int(current_x), int(current_y))
        
    circle_radius = 100
    color = (255, 0, 0)
    thickness = 10
    new_image = cv2.circle(new_image, center_coordinates, circle_radius, color, thickness)
    print(t,center_coordinates)
    return new_image

clip_fps = clip.fps
skipped_frame = None
num_of_skipped_frames = None
iterator = 0
traj1, = Trajectory.load_list('trackings/track_chaplin3.txt')

clip_blurred = clip.fl(object_tracked)

clip_blurred.ipython_display(loop=True, autoplay=True, rate=5000, width = 400)